In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage


import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')
import snapshot as snap
import plot as plot

VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'

In [ ]:
snap_nums = np.array([4, 9, 21, 39, 54, 68, 94])
Years = np.array(['4,000 yr', '9,000 yr', '21,000 yr', '39,000 yr', '54,000 yr', '68,000 yr', '94,000 yr'])
Masses = np.array(['1.4 $M_\odot$', '2 $M_\odot$', '4 $M_\odot$', '8 $M_\odot$', '12 $M_\odot$', '16 $M_\odot$', '24 $M_\odot$'])

# snap_nums = np.array([9, 21, 39, 54, 68, 94])
# Years = np.array(['9,000 yr', '21,000 yr', '39,000 yr', '54,000 yr', '68,000 yr', '94,000 yr'])
# Masses = np.array(['2 M$_\odot$', '4 M$_\odot$', '8 M$_\odot$', '12 M$_\odot$', '16 M$_\odot$', '24 $M_\odot$'])

Snapshots = np.zeros_like(snap_nums, dtype=snap.snapshot)

for ss in range(len(snap_nums)):
    name = 'Snap%03d_n' % snap_nums[ss]
    print(name, Masses[ss], Years[ss])
    Snapshots[ss] = snap.snapshot(snap=snap_nums[ss], name=name, read_zeusmp=False)
    Snapshots[ss].load_shock_variables()
    Snapshots[ss].load_fluxes()


In [ ]:
z_segments = np.array([0, 43, 66, 81, 92, 100, 108, 114, 119, 124, 
                       128, 132, 135, 139, 142, 145, 147, 150, 152, 154, 
                       156, 159, 160, 162, 164, 166, 167])
z_values = np.zeros(len(z_segments))
for i in range(len(z_segments)):
    z_values[i] = Snapshots[0].x1[z_segments[i]]
# print('z segment boundaries:', z_values)
xsquare = np.array([111, 169])
xsquarevals = np.zeros(2)
for j in range(2):
    xsquarevals[j] = Snapshots[0].x2[xsquare[j]]
xslice = np.array([0, len(Snapshots[0].x2)])

ionfrac_heights = np.zeros(len(z_segments)-1)
for i in range(len(ionfrac_heights)):
    ionfrac_heights[i] = np.average([z_values[i], z_values[i+1]])
# print('heights:', ionfrac_heights)
    
    
    
v_mins = np.array([1, 10, 100])


In [ ]:
def draw_ionfrac_xslice(fig, ax, Snap, Mass, 
                     rf_heights = np.array([3000, 5000, 9000, 18000]),
                     rf_fracs = np.array([.12, .17, .07, .05]),
                     rf_uplims = np.array([0,1,0,0], dtype=bool),
                     rf_yerr = np.array([.06, .06, .01, .01]),
                     show_legend=True, leg_cols=1, leg_anchor=(0,0.05), leg_loc='lower left'):

    ax.text(0.98, 0.02, Mass, ha='right', va='bottom', transform=ax.transAxes, fontsize=12, weight='bold')
#     ax.set_title(Mass + ' Ionization Fraction vs. Height')
    ax.set_yscale('log')
    # ax.set_ylim(ylim)
    #     if hide_labels is False:
    m100, = ax.plot(ionfrac_heights[:-1], Snap.mass_ionfrac_xslice[2,:-1], label='mass, 100km/s', 
                linestyle='solid', linewidth=3.5, color='blueviolet')
    m010, = ax.plot(ionfrac_heights[:-1], Snap.mass_ionfrac_xslice[1,:-1], label='mass, 10km/s', 
                linestyle='solid', linewidth=2.5, color='m')
    m001, = ax.plot(ionfrac_heights[:-1], Snap.mass_ionfrac_xslice[0,:-1], label='mass, 1km/s', 
                linestyle='solid', linewidth = 1.5, color='fuchsia')
    
    v100, = ax.plot(ionfrac_heights[:-1], Snap.vol_ionfrac_xslice[2,:-1], label='vol, 100km/s', 
                linestyle='dashed', linewidth=3.5, c='darkgreen')
    v010, = ax.plot(ionfrac_heights[:-1], Snap.vol_ionfrac_xslice[1,:-1], label='vol, 10km/s', 
                linestyle='dashed', linewidth=2.5, c='mediumseagreen')
    v001, = ax.plot(ionfrac_heights[:-1], Snap.vol_ionfrac_xslice[0,:-1], label='vol, 1km/s', 
                linestyle='dashed', linewidth=1.5, c='chartreuse')
    
    e100, = ax.plot(ionfrac_heights[:-1], Snap.emis_ionfrac_xslice[2,:-1], label='emis, 100km/s', 
                linestyle='dashdot', linewidth = 3.5, c='darkblue')
    e010, = ax.plot(ionfrac_heights[:-1], Snap.emis_ionfrac_xslice[1,:-1], label='emis, 10km/s', 
                linestyle='dashdot', linewidth = 2.5, c='blue')
    e001, = ax.plot(ionfrac_heights[:-1], Snap.emis_ionfrac_xslice[0,:-1], label='emis, 1km/s', 
                linestyle='dashdot', linewidth = 1.5, c='cornflowerblue')
    
    rf = ax.errorbar(rf_heights, rf_fracs, yerr=rf_yerr, uplims=rf_uplims, label='G35.2-0.74N',
                        marker = 'o', markersize=5, color='black', linestyle='none', capsize=5)
    
    if show_legend:
        leg = fig.legend(bbox_to_anchor=leg_anchor, loc=leg_loc, ncols=leg_cols, fontsize=14,
                         handles = [m100, m010, m001, v100, v010, v001, e100, e010, e001, rf,])

    return 

# Plot flooded (no cooling)

In [ ]:
def load_ionfrac_profile(Snap):
    data_path = (VICO_loc+'/Data/'+Snap.name+'/')
    loaded_ionfrac_profile= np.load((data_path+Snap.name+'_ionfrac_profile.npz'))
    Snap.mass_ionfrac_xsquare = loaded_ionfrac_profile['mass_ionfrac_xsquare']
    Snap.mass_ionfrac_xslice =  loaded_ionfrac_profile['mass_ionfrac_xslice']
    Snap.vol_ionfrac_xsquare = loaded_ionfrac_profile['vol_ionfrac_xsquare']
    Snap.vol_ionfrac_xslice = loaded_ionfrac_profile['vol_ionfrac_xslice']
    Snap.emis_ionfrac_xsquare =  loaded_ionfrac_profile['emis_ionfrac_xsquare']
    Snap.emis_ionfrac_xslice = loaded_ionfrac_profile['emis_ionfrac_xslice']
    print(Snap, 'ionfrac_profile loaded')

In [ ]:
xscale='linear'
ratio_label = 'no cooling'

fig, axs = plot.figax_double(height=7, nrows=2, ncols=4, xscale=xscale)
for ss in range(len(Snapshots)):
    load_ionfrac_profile(Snapshots[ss])
    rr = int(ss/4)
    cc = ss%4
    if ss == len(Snapshots)-1: show_legend=True
    else: show_legend=False
    draw_ionfrac_xslice(fig, axs[rr,cc], Snapshots[ss], Masses[ss], show_legend=show_legend,
                        leg_cols=1, leg_anchor=(0.97, 0.49), leg_loc='upper right')


# set up axes labels and limits
ylim = (10**-4, 10**0)


axs[1,0].set_xticks([0, 5000, 10000, 15000, 20000, 25000])
for ax in axs[1,1:3]:
    ax.set_xticks([ 5000, 10000, 15000, 20000, 25000])
for ax in axs[0,:3]:
    ax.set_xticks([])
for ax in axs[0,:]:
    ax.set_xlim(axs[1,0].get_xlim())
for ax in axs.flatten():
    ax.set_ylim(ylim)
axs[0,0].set_yticks([10**-3, 10**-2, 10**-1, 10**0])
axs[1,0].set_yticks([10**-4, 10**-3, 10**-2, 10**-1, 10**0])
for ax in axs[:,1:4].flatten():
    ax.set_yticks([])

fig.text(0.5, 0.04, '$z$ (au)', ha='center', va='center', fontsize=plot.FONTSIZE)


fig.text(0.04, 0.5, r'$\langle\chi_\mathrm{H+}\rangle$', ha='center', va='center', 
                rotation='vertical', fontsize=plot.FONTSIZE)
axs[1,3].remove()
fig.subplots_adjust(wspace=0, hspace=0)

# fig.legend(bbox_to_anchor=(0.05,0), loc='upper left', ncols=4, fontsize=14)
# fig.text(1.0,0.97, ratio_label, fontsize=14, 
        #  weight='bold', horizontalalignment='left', verticalalignment='top')

fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/ionfrac_profile_caseA.png', 
            dpi=200, bbox_inches='tight')

    

# Plot ratio (with cooling)

In [ ]:
def load_ionfrac_ratio_profile(Snap):
    data_path = (VICO_loc+'/Data/'+Snap.name+'/')
    loaded_ionfrac_profile= np.load((data_path+Snap.name
                        +'_ionfrac_ratio_profile.npz'))
    Snap.mass_ionfrac_xsquare = \
    loaded_ionfrac_profile['mass_ionfrac_xsquare']
    Snap.mass_ionfrac_xslice =  \
    loaded_ionfrac_profile['mass_ionfrac_xslice']
    Snap.vol_ionfrac_xsquare = \
    loaded_ionfrac_profile['vol_ionfrac_xsquare']
    Snap.vol_ionfrac_xslice = \
    loaded_ionfrac_profile['vol_ionfrac_xslice']
    Snap.emis_ionfrac_xsquare =  \
    loaded_ionfrac_profile['emis_ionfrac_xsquare']
    Snap.emis_ionfrac_xslice = \
    loaded_ionfrac_profile['emis_ionfrac_xslice']
    print(Snap, 'ionfrac_profile loaded')

for ss in range(len(Snapshots)):
    load_ionfrac_ratio_profile(Snapshots[ss])

In [ ]:
xscale='linear'
ratio_label = 'no cooling'

fig, axs = plot.figax_double(height=7, nrows=2, ncols=4, xscale=xscale)
for ss in range(len(Snapshots)):
    load_ionfrac_ratio_profile(Snapshots[ss])
    rr = int(ss/4)
    cc = ss%4
    if ss == len(Snapshots)-1: show_legend=True
    else: show_legend=False
    draw_ionfrac_xslice(fig, axs[rr,cc], Snapshots[ss], Masses[ss], show_legend=show_legend,
                        leg_cols=1, leg_anchor=(0.97, 0.49), leg_loc='upper right')


# set up axes labels and limits
ylim = (10**-4, 10**0)


axs[1,0].set_xticks([0, 5000, 10000, 15000, 20000, 25000])
for ax in axs[1,1:3]:
    ax.set_xticks([ 5000, 10000, 15000, 20000, 25000])
for ax in axs[0,:3]:
    ax.set_xticks([])
for ax in axs[0,:]:
    ax.set_xlim(axs[1,0].get_xlim())
for ax in axs.flatten():
    ax.set_ylim(ylim)
axs[0,0].set_yticks([10**-3, 10**-2, 10**-1, 10**0])
axs[1,0].set_yticks([10**-4, 10**-3, 10**-2, 10**-1, 10**0])
for ax in axs[:,1:4].flatten():
    ax.set_yticks([])

fig.text(0.5, 0.04, '$z$ (au)', ha='center', va='center', fontsize=plot.FONTSIZE)


fig.text(0.04, 0.5, r'$\langle\chi_\mathrm{H+}\rangle$', ha='center', va='center', 
                rotation='vertical', fontsize=plot.FONTSIZE)
axs[1,3].remove()
fig.subplots_adjust(wspace=0, hspace=0)

# fig.legend(bbox_to_anchor=(0.05,0), loc='upper left', ncols=4, fontsize=14)
# fig.text(1.0,0.97, ratio_label, fontsize=14, 
        #  weight='bold', horizontalalignment='left', verticalalignment='top')

fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/ionfrac_profile_caseB.png', 
            dpi=200, bbox_inches='tight')

    

In [ ]:
xscale='log'
ratio_label='with cooling'

fig, axs = plot.figax(figsize=(12,8), nrows=2, ncols=3, sharex=True, sharey=True, xscale=xscale)
for ss in range(len(Snapshots)):
    load_ionfrac_ratio_profile(Snapshots[ss])
    rr = int(ss/3)
    cc = ss%3
    if ss == len(Snapshots)-1: show_legend=True
    else: show_legend=False
    draw_ionfrac_xslice(fig, axs[rr,cc], Snapshots[ss], Masses[ss], show_legend=show_legend,
                        leg_cols=1, leg_anchor=(0.99, 0.94), leg_loc='upper left',
                        ylim=(10**-4, 10**0))
    if rr==1: axs[rr,cc].set_xlabel('$z$ (au)', fontsize=14)
    if cc==0: axs[rr,cc].set_ylabel(r'$\langle\chi_\mathrm{H+}\rangle$', fontsize=14)
fig.text(1.0,0.97, ratio_label, fontsize=14, 
         weight='bold', horizontalalignment='left', verticalalignment='top')
fig.tight_layout()
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/ionfrac_profile_ratio_log.png', 
            dpi=300, bbox_inches='tight')
    

In [ ]:
xscale='linear'
ratio_label='with cooling'

fig, axs = plot.figax(figsize=(12,8), nrows=2, ncols=3, sharex=True, sharey=True, xscale=xscale)
for ss in range(len(Snapshots)):
    load_ionfrac_ratio_profile(Snapshots[ss])
    rr = int(ss/3)
    cc = ss%3
    if ss == len(Snapshots)-1: show_legend=True
    else: show_legend=False
    draw_ionfrac_xslice(fig, axs[rr,cc], Snapshots[ss], Masses[ss], show_legend=show_legend,
                        leg_cols=1, leg_anchor=(0.99, 0.94), leg_loc='upper left',
                        ylim=(10**-4, 10**0))
    if rr==1: axs[rr,cc].set_xlabel('$z$ (au)', fontsize=14)
    if cc==0: axs[rr,cc].set_ylabel(r'$\langle\chi_\mathrm{H+}\rangle$', fontsize=14)
fig.text(1.0,0.97, ratio_label, fontsize=14, 
         weight='bold', horizontalalignment='left', verticalalignment='top')
# fig.legend(bbox_to_anchor=(0.05,0), loc='upper left', ncols=4, fontsize=14)
fig.tight_layout()
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/ionfrac_profile_ratio_lin.png', 
            dpi=300, bbox_inches='tight')

    